## Installera Tensorflow och Tensorflow.hub

In [ ]:
# Du behöver bara köra den här cellen en gång
!pip install tensorflow
!pip install tensorflow_hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Plotfunktioner - plottar bilderna

In [ ]:
def img_in_plot(content_image, style_image):

    plt.figure(figsize=(12, 12))
    plt.subplot(1, 2, 1)
    plt.imshow(content_image)
    plt.title('Content Image')
    plt.subplot(1, 2, 2)
    plt.imshow(style_image)
    plt.title('Style Image')

    plt.show()

In [ ]:
def img_out_plot(stylized_image):
    # Set the size of the plot figure
    plt.figure(figsize=(12, 12))

    # Plot the stylized image
    plt.imshow(stylized_image[0])

    # Add title to the plot
    plt.title('Stylized Image')

    # Hide axes
    plt.axis('off')

    # Show the plot
    plt.show()
  

# Style transfer av en bild
Exempelkod från
https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2

In [ ]:
# Funktionen style_transfer är samma som exempelkoden i tfhub men med plottning av bilderna
def style_transfer(content_path, style_path):
     # Load content and style images (see example in the attached colab).
    content_image = plt.imread(content_path)
    style_image = plt.imread(style_path)
    img_in_plot(content_image, style_image)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    # Optionally resize the images. It is recommended that the style image is about
    # 256 pixels (this size was used when training the style transfer network).
    # The content image can be any size.
    style_image = tf.image.resize(style_image, (256, 256))

    # Stylize image.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0] 
    img_out_plot(stylized_image)

## Ladda arbitrary-image-stylization

In [ ]:
# Om fel "tf hub OSError: SavedModel file does not exist at" ta bort foldern och kör om...
# /1 eller /2 (del av versionen) ger olika resultat. Jag föredrar /1 men pröva gärna /2

# Load Magenta's Arbitrary Image Stylization network from TensorFlow Hub  
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')

# Våra datastrukturer

In [ ]:
def style_transfer_data(content_path, style_path):
     # Load content and style images (see example in the attached colab).
    content_image = plt.imread(content_path)
    print("Bild efter inläsning (content_image):")
    print(type(content_image))
    print(content_image.shape)
    style_image = plt.imread(style_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.
    print("Bild efter transformering (content_image):")
    print(type(content_image))
    print(content_image.shape)    
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    # Optionally resize the images. It is recommended that the style image is about
    # 256 pixels (this size was used when training the style transfer network).
    # The content image can be any size.
    style_image = tf.image.resize(style_image, (256, 256))

    # Stylize image.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0] 
    print("Resultat-bild (stylized_image):")
    print(type(stylized_image))
    print(stylized_image.shape)

In [ ]:
style_transfer_data(Path("data") / "blue-cheer.jpg",
               Path("data") / "hilma-af-klint.JPG")

## Pröva olika stilar (style) för vår bild (content)
För att slå av/på scrollning inom cell: Klicka till vänster i cellen men utanför bilden

In [ ]:
style_transfer(Path("data") / "blue-cheer.jpg",
               Path("data") / "hilma-af-klint.JPG")

In [ ]:
style_transfer(Path("data") / "blue-cheer.jpg",
              Path("data") / "van_gogh_bedroom_in_arles.jpg")

In [ ]:
style_transfer(Path("data") / "blue-cheer.jpg",
               Path("data") / "john-bauer-troll.JPG")

# Funktioner för style transfer av videoclip

In [ ]:
# Style transfer av en frame (bild) 
def style_transfer_frame(content_path, style_image, target_path):
    # Load content image
    content_image = plt.imread(content_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.

    # Stylize image.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0]
    tf.keras.utils.save_img(target_path, stylized_image[0])

In [ ]:
# Style transfer av ett clip
def style_transfer_clip(end, content_path_base, style_path, target_path_base):
    from pathlib import Path
    # Load content style image
    style_image = plt.imread(style_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    # Optionally resize the images. It is recommended that the style image is about
    # 256 pixels (this size was used when training the style transfer network).
    # The content image can be any size.
    style_image = tf.image.resize(style_image, (256, 256))
    for i in range(1, end + 1):
        path_end = "scene{:05d}.jpg".format(i)
        style_transfer_frame(content_path_base / (path_end),
                            style_image,
                            target_path_base / (path_end))
        

# Style transfer för video från Blue Cheer
### OBS! Anropen av ffmpeg måste du ändra till var du har installerat ffmpeg men oxå path till filer i foldrarna. Pathlib funkar inte här...

In [ ]:
# Skapa bilderna från videon. Här för den första minuten.
!d:\ffmpeg\bin\ffmpeg -ss 00:00 -i data\\blue-cheer.mp4 -t 01:00 -loglevel error clipin\\scene%05d.jpg

### Testa med stil Hilma af Klint

In [ ]:
# Style transfer på de första 200 bilderna
# För hela videon finns det 5518 bilder
# Det tar ungefär 1 sekund per bild
style_transfer_clip(200,
                Path("clipin"),
                Path("data") / "hilma-af-klint.jpg",
                Path("cliput"))

In [ ]:
# Skapa videon från bilderna
!d:\ffmpeg\bin\ffmpeg -start_number 00001 -i cliput\\scene%05d.jpg  -r 25 -y -loglevel error blue-cheer.mp4

In [ ]:
# Lägg på ljudet
!d:\ffmpeg\bin\ffmpeg -i blue-cheer.mp4 -i data\blue-cheer.m4a -map 0:v -map 1:a -c copy -shortest -y -loglevel error blue-cheer-1-ws.mp4

In [ ]:
# Visa videon med repeat i ett särskilt fönster
# Du kan oxå förstås visa videon från foldern AI-experiment\deep_dream och använda din vanliga video-spelare
!d:\ffmpeg\bin\ffplay -loop 5 -loglevel error blue-cheer-1-ws.mp4

### Testa med stil van Gogh

In [ ]:
# Style transfer på de första 200 bilderna
# För hela videon finns det 5518 bilder
# Det tar ungefär 1 sekund per bild
style_transfer_clip(200,
                Path("clipin"),
                Path("data") / "van_gogh_bedroom_in_arles.jpg",
                Path("cliput"))

In [ ]:
# Skapa videon från bilderna
!d:\ffmpeg\bin\ffmpeg -start_number 00001 -i cliput\\scene%05d.jpg  -r 25 -y -loglevel error blue-cheer.mp4

In [ ]:
# Lägg på ljudet
!d:\ffmpeg\bin\ffmpeg -i blue-cheer.mp4 -i data\blue-cheer.m4a -map 0:v -map 1:a -c copy -shortest -y -loglevel error blue-cheer-2-ws.mp4

In [ ]:
# Visa videon med repeat i ett särskilt fönster
# Du kan oxå förstås visa videon från foldern AI-experiment\deep_dream och använda din vanliga video-spelare
!d:\ffmpeg\bin\ffplay -loop 5 -loglevel error blue-cheer-2-ws.mp4

### Testa med stil John Bauer

In [ ]:
# Style transfer på de första 200 bilderna
# För hela videon finns det 5518 bilder
# Det tar ungefär 1 sekund per bild
style_transfer_clip(200,
                 Path("clipin"),
                 Path("data") / "john-bauer-troll.JPG",
                 Path("cliput"))

In [ ]:
# Skapa videon från bilderna
!d:\ffmpeg\bin\ffmpeg -start_number 00001 -i cliput\\scene%05d.jpg  -r 25 -y -loglevel error blue-cheer.mp4

In [ ]:
# Lägg på ljudet
!d:\ffmpeg\bin\ffmpeg -i blue-cheer.mp4 -i data\blue-cheer.m4a -map 0:v -map 1:a -c copy -shortest -y -loglevel error blue-cheer-3-ws.mp4

In [ ]:
# Visa videon med repeat i ett särskilt fönster
# Du kan oxå förstås visa videon från foldern AI-experiment\deep_dream och använda din vanliga video-spelare
!d:\ffmpeg\bin\ffplay -loop 5 -loglevel error blue-cheer-3-ws.mp4